In [1]:
from pyecharts import Geo, Bar, Line, Overlap, EffectScatter, Map

In [22]:
# 城市 -- 指定省的城市 xx市
city = ['延庆区', '怀柔区', '昌平区', '海淀区', '朝阳区', '顺义区', '平谷区', '密云区',
        '大兴区', '西城区', '东城区', '门头沟区', '房山区', '通州区', '丰台区', '石景山区']
values2 = [1.07, 3.85, 6.38, 8.21, 2.53, 4.37, 9.38,
           4.29, 6.1, 1.07, 3.85, 6.38, 8.21, 2.53, 4.37, 2.53]


# 河南地图  数据必须是省内放入城市名
map2 = Map("北京地图", '北京', width=1400, height=800)
map2.add('北京', city, values2, visual_range=[1, 10],
         maptype='北京', is_visualmap=True, visual_text_color='#000', is_map_symbol_show=False)
map2

In [ ]:
# 加载pyecharts
from pyecharts import Geo, Style
import pandas as pd

# 导入excel表举例
df = pd.read_excel('流动人口.xlsx')
df.head()

# 导入自定义的地点经纬度
geo_cities_coords = {df.iloc[i]['地点']: [df.iloc[i]['经度'], df.iloc[i]['纬度']]
                     for i in range(len(df))}  # 根据文件大小生成字典
attr = list(df['地点'])  # 字典的每个键值
value = list(df['流动人口']/100000)  # 由于量值的太大，换算以下(散点的颜色就是和这个想关的)
style = Style(title_color="#fff", title_pos="center",
              width=1200, height=600, background_color="#404a59")

# 可视化
geo = Geo('东莞各个CGI总用户数分布', **style.init_style)
geo.add("", attr, value, visual_range=[0, 100], symbol_size=5,
        visual_text_color="#fff", is_piecewise=True,
        is_visualmap=True, maptype='东莞', visual_split_number=10,
        geo_cities_coords=geo_cities_coords)

geo.render('东莞各个CGI总用户数分布.html')